In [1]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install imblearn
!pip install requests

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
TOKEN = "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkdYdExONzViZlZQakdvNERWdjV4QkRITHpnSSIsImtpZCI6IkdYdExONzViZlZQakdvNERWdjV4QkRITHpnSSJ9.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmZpcmVhbnQudm4iLCJhdWQiOiJodHRwczovL2FjY291bnRzLmZpcmVhbnQudm4vcmVzb3VyY2VzIiwiZXhwIjoyMDU0NDYzNzAwLCJuYmYiOjE3NTQ0NjM3MDAsImNsaWVudF9pZCI6ImZpcmVhbnQudHJhZGVzdGF0aW9uIiwic2NvcGUiOlsib3BlbmlkIiwicHJvZmlsZSIsInJvbGVzIiwiZW1haWwiLCJhY2NvdW50cy1yZWFkIiwiYWNjb3VudHMtd3JpdGUiLCJvcmRlcnMtcmVhZCIsIm9yZGVycy13cml0ZSIsImNvbXBhbmllcy1yZWFkIiwiaW5kaXZpZHVhbHMtcmVhZCIsImZpbmFuY2UtcmVhZCIsInBvc3RzLXdyaXRlIiwicG9zdHMtcmVhZCIsInN5bWJvbHMtcmVhZCIsInVzZXItZGF0YS1yZWFkIiwidXNlci1kYXRhLXdyaXRlIiwidXNlcnMtcmVhZCIsInNlYXJjaCIsImFjYWRlbXktcmVhZCIsImFjYWRlbXktd3JpdGUiLCJibG9nLXJlYWQiLCJpbnZlc3RvcGVkaWEtcmVhZCJdLCJzdWIiOiIzOTViODBjZS1lY2M0LTRhMDktYTc5YS0yM2YyMDRiOWM2ODEiLCJhdXRoX3RpbWUiOjE3NTQ0NjM3MDAsImlkcCI6Ikdvb2dsZSIsIm5hbWUiOiJodWZsaXQuY2xhc3Nyb29tQGdtYWlsLmNvbSIsInNlY3VyaXR5X3N0YW1wIjoiOGU0MDQxOTktODVhMi00YjI0LWE1NTItZmMyMGE2NjM4Y2FmIiwianRpIjoiYWYzNjIzZTgzYmFhZTk0OTBhYjZiYTg1ZTg0OGJlNjgiLCJhbXIiOlsiZXh0ZXJuYWwiXX0.fVqhDYtLHVJDN8EqSacMpDwm3m7VlBQuloJ2Qe6VeWu39c_aplpcu50rJTsZn09dUX00B16mhBpgV87NEGQu7kuOObxNY25IqDqtHSIvAX6u7tn9B_8ZY0CSNoPdT1XxHWEaz7um2dXugj2nKG71XROGjvfiZMIjJkzq_noK2Am2iuCn2bYFRBfwmX7ILXqyRq_3SOkWGVcLYPyNLS3LeVC77CubfhH29JnQiuX4migR7uKxvXdQHo48BdeumgGac_BBmwJHpBlwm_PSNy_8lREVvJgHSf6n-sa2vTHaOddxVzDga5KIsYpQzs3Cvhs3WRooCKhCPXXAOTV6PKAW3g"
csv_file = "listcp.csv"

In [ ]:
import requests
import pandas as pd
import json
import time
import os

def download_stock_list(csv_file, token):
    # 1. Đọc danh sách mã từ file CSV
    # Giả sử file CSV chỉ có 1 cột chứa mã
    try:
        df_list = pd.read_csv(csv_file, header=None)
        symbols = df_list[0].tolist()
    except Exception as e:
        print(f"Lỗi khi đọc file CSV: {e}")
        return

    headers = {
        "User-Agent": "Mozilla/5.0",
        "Authorization": f"Bearer {token}"
    }

    # 2. Vòng lặp duyệt qua từng mã
    for symbol in symbols:
        symbol = symbol.strip().upper() # Làm sạch dữ liệu mã
        if not symbol: continue

        all_data = []
        offset = 0
        limit = 1000
        url = f"https://restv2.fireant.vn/symbols/{symbol}/historical-quotes"

        print(f"\n--- Đang tải mã: {symbol} ---")

        while True:
            params = {
                "startDate": "2000-01-01",
                "endDate": "2026-12-31",
                "offset": offset,
                "limit": limit
            }

            try:
                response = requests.get(url, params=params, headers=headers)
                if response.status_code != 200:
                    print(f"  [!] Lỗi {response.status_code} khi tải {symbol}")
                    break

                data = response.json()
                if not data: break # Hết dữ liệu

                for item in data:
                    all_data.append({
                        "time": item['date'].split('T')[0],
                        "open": item['priceOpen'],
                        "high": item['priceHigh'],
                        "low": item['priceLow'],
                        "close": item['priceClose'],
                        "value": item['totalVolume'],
                        "basic": item['priceBasic'],	#priceBasic
                        "dvolume": item['dealVolume'],	#dealVolume
                        "tvalue": item['totalValue'],	#totalValue
                        "ptvalue": item['putthroughValue'],	#putthroughValue
                        "bfquantity": item['buyForeignQuantity'],	#buyForeignQuantity
                        "bfvalue": item['buyForeignValue'],	#buyForeignValue
                        "sfquantity": item['sellForeignQuantity'],	#sellForeignQuantity
                        "sfvalue": item['sellForeignValue'],	#sellForeignValue
                        "bcount": item['buyCount'],	#buyCount
                        "bquantity": item['buyQuantity'],	#buyQuantity
                        "scount": item['sellCount'],	#sellCount
                        "squantity": item['sellQuantity'],	#propTradingNetDealValue
                        "aratio": item['adjRatio'],	#propTradingNetDealValue
                        "cfroom": item['currentForeignRoom'],	#propTradingNetDealValue
                        "ptndvalue": item['propTradingNetDealValue'],	#propTradingNetDealValue
                        "ptnetptvalue": item['propTradingNetPTValue'],	#propTradingNetPTValue
                        "ptnvalue": item['propTradingNetValue'],	#propTradingNetValue
                        "unit": item['unit'],	#unit
                    })

                if len(data) < limit: break # Trang cuối cùng
                offset += limit
                time.sleep(0.3) # Nghỉ ngắn để tránh bị chặn

            except Exception as e:
                print(f"  [!] Lỗi phát sinh: {e}")
                break

        # 3. Lưu dữ liệu của mã này vào file JSON
        if all_data:
            # Sắp xếp theo thời gian tăng dần
            all_data.sort(key=lambda x: x['time'])

            filename = f"{symbol}.json"
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(all_data, f, indent=4)
            print(f"  [✓] Hoàn tất: {len(all_data)} nến -> {filename}")
        else:
            print(f"  [x] Không có dữ liệu cho mã {symbol}")

# --- THỰC THI ---

download_stock_list('listcp.csv', TOKEN)

# 1. Nén tất cả các file JSON thành file ZIP
!zip -r all_stocks.zip *.json

# 2. Lệnh tự động tải file về máy
from google.colab import files
files.download('all_stocks.zip')

Streaming output truncated to the last 5000 lines.
--- Đang tải mã: GTS ---
  [✓] Hoàn tất: 2300 nến -> GTS.json

--- Đang tải mã: GTT ---
  [✓] Hoàn tất: 3868 nến -> GTT.json

--- Đang tải mã: GVR ---
  [✓] Hoàn tất: 1941 nến -> GVR.json

--- Đang tải mã: GVT ---
  [✓] Hoàn tất: 2242 nến -> GVT.json

--- Đang tải mã: H11 ---
  [✓] Hoàn tất: 3667 nến -> H11.json

--- Đang tải mã: HAC ---
  [✓] Hoàn tất: 4521 nến -> HAC.json

--- Đang tải mã: HAD ---
  [✓] Hoàn tất: 4041 nến -> HAD.json

--- Đang tải mã: HAF ---
  [✓] Hoàn tất: 2113 nến -> HAF.json

--- Đang tải mã: HAG ---
  [✓] Hoàn tất: 4250 nến -> HAG.json

--- Đang tải mã: HAH ---
  [✓] Hoàn tất: 2705 nến -> HAH.json

--- Đang tải mã: HAI ---
  [✓] Hoàn tất: 4723 nến -> HAI.json

--- Đang tải mã: HAM ---
  [✓] Hoàn tất: 2093 nến -> HAM.json

--- Đang tải mã: HAN ---
  [✓] Hoàn tất: 2301 nến -> HAN.json

--- Đang tải mã: HAP ---
  [✓] Hoàn tất: 6013 nến -> HAP.json

--- Đang tải mã: HAR ---
  [✓] Hoàn tất: 3232 nến -> HAR.json

--- 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 1. Nén tất cả các file JSON thành file ZIP
!zip -r all_stocks.zip *.json

# 2. Lệnh tự động tải file về máy
from google.colab import files
files.download('all_stocks.zip')

updating: A32.json (deflated 96%)
updating: AAA.json (deflated 90%)
updating: AAM.json (deflated 94%)
updating: AAS.json (deflated 91%)
updating: AAT.json (deflated 91%)
updating: AAV.json (deflated 92%)
updating: ABB.json (deflated 91%)
updating: ABC.json (deflated 94%)
updating: ABI.json (deflated 93%)
updating: ABR.json (deflated 95%)
updating: ABS.json (deflated 92%)
updating: ABT.json (deflated 93%)
updating: ACB.json (deflated 91%)
updating: ACC.json (deflated 93%)
updating: ACE.json (deflated 95%)
updating: ACG.json (deflated 92%)
updating: ACL.json (deflated 92%)
updating: ACM.json (deflated 94%)
updating: ACS.json (deflated 97%)
updating: ACV.json (deflated 88%)
updating: ADC.json (deflated 96%)
updating: ADG.json (deflated 92%)
updating: ADP.json (deflated 96%)
updating: ADS.json (deflated 92%)
updating: AFX.json (deflated 94%)
updating: AG1.json (deflated 96%)
updating: AGE.json (deflated 98%)
updating: AGF.json (deflated 95%)
updating: AGG.json (deflated 89%)
updating: AGM.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Intraday

In [5]:
import requests
import json
import os
import pandas as pd
from datetime import datetime

def update_stock_data(symbol, timeframe, token):
    # 1. Cấu hình lấy dữ liệu (Chỉ lấy ngày hôm nay)
    today_str = datetime.now().strftime('%Y-%m-%d')
    res_map = {'5m': '5', '1h': '60', '1d': '1D'}
    res = res_map.get(timeframe, '1D')
    
    url = f"https://restv2.fireant.vn/symbols/{symbol}/historical-quotes"
    headers = {"Authorization": f"Bearer {token}", "User-Agent": "Mozilla/5.0"}
    
    # Chỉ lấy dữ liệu trong khoảng 3 ngày gần nhất để đảm bảo không sót nến
    params = {
        "startDate": "2025-12-30", # Thay đổi tùy theo thời điểm bạn chạy
        "endDate": today_str,
        "resolution": res,
        "offset": 0,
        "limit": 500
    }

    try:
        response = requests.get(url, params=params, headers=headers)
        if response.status_code != 200:
            print(f"Lỗi API mã {symbol}: {response.status_code}")
            return
        
        new_data_raw = response.json()
        if not new_data_raw:
            print(f"Không có dữ liệu mới cho {symbol}")
            return

        # Định dạng dữ liệu mới
        new_data = []
        for item in new_data_raw:
            new_data.append({
                "time": item['date'].split('T')[0],
                "open": item['priceOpen'],
                "high": item['priceHigh'],
                "low": item['priceLow'],
                "close": item['priceClose'],
                "value": item['totalVolume'],
                "basic": item['priceBasic'],	#priceBasic
                "dvolume": item['dealVolume'],	#dealVolume
                "tvalue": item['totalValue'],	#totalValue
                "ptvalue": item['putthroughValue'],	#putthroughValue
                "bfquantity": item['buyForeignQuantity'],	#buyForeignQuantity
                "bfvalue": item['buyForeignValue'],	#buyForeignValue
                "sfquantity": item['sellForeignQuantity'],	#sellForeignQuantity
                "sfvalue": item['sellForeignValue'],	#sellForeignValue
                "bcount": item['buyCount'],	#buyCount
                "bquantity": item['buyQuantity'],	#buyQuantity
                "scount": item['sellCount'],	#sellCount
                "squantity": item['sellQuantity'],	#propTradingNetDealValue
                "aratio": item['adjRatio'],	#propTradingNetDealValue
                "cfroom": item['currentForeignRoom'],	#propTradingNetDealValue
                "ptndvalue": item['propTradingNetDealValue'],	#propTradingNetDealValue
                "ptnetptvalue": item['propTradingNetPTValue'],	#propTradingNetPTValue
                "ptnvalue": item['propTradingNetValue'],	#propTradingNetValue
                "unit": item['unit'],	#unit
            })

        # 2. Đọc file cũ và gộp dữ liệu
        filename = f"./data/{symbol.upper()}.json"
        existing_data = []
        
        if os.path.exists(filename):
            with open(filename, 'r', encoding='utf-8') as f:
                existing_data = json.load(f)
        
        # Gộp dữ liệu mới vào dữ liệu cũ
        combined_data = existing_data + new_data

        # 3. Lọc trùng lặp (Dựa trên trường 'time')
        # Quan trọng để tránh việc chạy code 2 lần bị nhân đôi nến
        seen = set()
        unique_data = []
        for d in combined_data:
            if d['time'] not in seen:
                unique_data.append(d)
                seen.add(d['time'])

        # 4. Sắp xếp lại theo thời gian
        unique_data.sort(key=lambda x: x['time'])

        # 5. Lưu lại vào file
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(unique_data, f, indent=4)
        
        print(f"✓ Đã cập nhật xong {symbol}. Tổng: {len(unique_data)} nến.")

    except Exception as e:
        print(f"Lỗi xử lý {symbol}: {e}")

def update_All(csv_file, token):
    # --- THỰC THI CHO DANH SÁCH ---
    # 1. Đọc danh sách mã từ file CSV
    # Giả sử file CSV chỉ có 1 cột chứa mã
    try:
        df_list = pd.read_csv(csv_file, header=None)
        symbols = df_list[0].tolist()
    except Exception as e:
        print(f"Lỗi khi đọc file CSV: {e}")
        return


    # 2. Update tung co phieu
    for cp in symbols:
        update_stock_data(cp, '1d', token) # Cập nhật khung ngày
        # update_stock_data(cp, '5m', TOKEN) # Nếu muốn cập nhật khung 5p

update_All(csv_file,TOKEN)

✓ Đã cập nhật xong A32. Tổng: 1808 nến.
✓ Đã cập nhật xong AAA. Tổng: 3866 nến.
✓ Đã cập nhật xong AAM. Tổng: 4072 nến.
✓ Đã cập nhật xong AAS. Tổng: 1372 nến.
✓ Đã cập nhật xong AAT. Tổng: 1205 nến.
✓ Đã cập nhật xong AAV. Tổng: 1893 nến.
✓ Đã cập nhật xong ABB. Tổng: 1261 nến.
✓ Đã cập nhật xong ABC. Tổng: 2355 nến.
✓ Đã cập nhật xong ABI. Tổng: 4114 nến.
✓ Đã cập nhật xong ABR. Tổng: 1895 nến.
✓ Đã cập nhật xong ABS. Tổng: 1460 nến.
✓ Đã cập nhật xong ABT. Tổng: 4740 nến.
✓ Đã cập nhật xong ACB. Tổng: 4774 nến.
✓ Đã cập nhật xong ACC. Tổng: 4004 nến.
✓ Đã cập nhật xong ACE. Tổng: 4037 nến.
✓ Đã cập nhật xong ACG. Tổng: 1105 nến.
✓ Đã cập nhật xong ACL. Tổng: 4331 nến.
✓ Đã cập nhật xong ACM. Tổng: 2618 nến.
✓ Đã cập nhật xong ACS. Tổng: 2129 nến.
✓ Đã cập nhật xong ACV. Tổng: 2289 nến.
✓ Đã cập nhật xong ADC. Tổng: 3759 nến.
✓ Đã cập nhật xong ADG. Tổng: 1503 nến.
✓ Đã cập nhật xong ADP. Tổng: 3935 nến.
✓ Đã cập nhật xong ADS. Tổng: 2389 nến.
✓ Đã cập nhật xong AFX. Tổng: 2274 nến.
